### まずはライブラリのインポート

In [35]:
# Load class room dump.
import pickle

# Regex
import re

# Import class room
from class_rooms import ClassRoom

# pandas セット
import pandas as pd
from pandas import DataFrame
import numpy as np

# pip install janome
from janome.tokenizer import Tokenizer

### 勉強会データの読み込みと、データ加工の準備

勉強会の名称バックアップを読み込んで

In [36]:
# load class rooms
class_rooms = []
with open('backup.pickle', 'rb') as f:
    class_rooms.extend(pickle.load(f))

品詞分解後の格納リストを用意する

In [37]:
# to lists
tokenized_list = []

# Tokenized
cur_idx = 0
tokenizer = Tokenizer()

記録したく無い名前（ここでは「数字記号のみ」「テコンドー」「練習」みたいな名詞は無視＋一文字はキーワードと認めない）を決め打ちする。  
これだけだと、R言語とかが割を食うけど、まぁ今回は諦め。

形態素解析しながらリストを作成していく（ただし、一つの勉強会タイトル中に複数回同じ単語が出たらカウントしない）。

In [38]:
# ただしここで列挙されるような単語は記録しない
remove_names = ['^[0-9\/\(\)\!]+$', '[#\*]', 'テコンドー', '練習']
def is_valid(value: str):
    if len(value) <= 1:
        return False
    for r in remove_names:
        if re.match(r, value):
            return False
    return True
    

# 勉強会を形態素解析して、名詞単位の行に分割
for cr in class_rooms:
    tokens = tokenizer.tokenize(cr.title)  # 形態素解析
    already = []  # 既出名詞

    for token in tokens:
        pos = token.part_of_speech.split(',')[0]  # 品詞名取得

        if ('名詞' in pos) and (token.surface not in already) and is_valid(token.surface):
            # 新出の名詞なら
            already.append(token.surface)
            tokenized_list.append(
                [cur_idx, token.surface, cr.joins, cr.limit])
    cur_idx += 1

リスト化したら、ヘッダをつけて Pandas に食わせ、DataFrame に変換する。

In [39]:
# Pandas DataFrame 変換
cr_df = DataFrame(data=tokenized_list, columns=['index', 'surface', 'joins', 'limit'])
cr_df

,index,surface,joins,limit
0,0,体験,0,5
1,0,歓迎,0,5
2,0,自分,0,5
3,0,可能,0,5
4,0,東京,0,5
5,0,下北沢,0,5
6,1,パルフェ,5,120
7,1,閉館,5,120
8,1,ラスト,5,120
9,1,イベント,5,120


### 簡単にキーワード抽出

まずは際頻出キーワード集計する。  
色々行なっているけど、行ごとに説明すると

1. キーワードでグループ化＋集計する。  
   この時点で、「キー:件数」の pandas.Series 型に変換される。
2. カウント数がこのままだと加工しづらいので、DataFrame に変換する。  
   reset_index をしてやることで、インデックス行が自動で作成され、ちゃんと合計数に count のラベルがつく
3. count が 10 以上の物を、逆順ソートで取得

In [41]:
group_sizes = cr_df.groupby('surface').size()
gs_df = DataFrame({ 'count': group_sizes }).reset_index()
gs_df[gs_df['count'] > 10].sort_values('count', ascending = False)

,surface,count
1319,勉強,102
746,もくもく,71
625,in,66
947,セミナー,57
1277,入門,55
1916,開催,50
1357,向け,49
1610,東京,48
1303,初心者,44
110,CoderDojo,41


この中から技術単語を拾うと

1. AI : 40
2. Python : 40
3. デザイン : 19
4. Web : 19
5. 機械(学習) : 14
6. 統計 : 13
7. Clowd : 11
8. RPA : 11

勉強会の開催頻度だけで見ると、AI(with Python)がダントツ。  
どれだけ注目されているかがよくわかる。

ちなみに、「機械」に関しても以下を見ればわかるが、全て機械学習。  
AI と被ってる勉強会は 3 件だけなので、重複排除で見れば

1. AI : 50
2. Python : 40
3. デザイン : 19
4. Web : 19
5. 統計 : 13
6. Clowd : 11
7. RPA : 11

In [43]:
for cr in class_rooms:
    if '機械' in cr.title:
        print(cr)

ClassRoom(【実務で使える機械学習講座】経験豊富なデータサイエンティストが基礎から実務レベルまで教えます, 20, 0)
ClassRoom(【初参加者向け】【少人数制】AI・機械学習と数学・プログラミングの学び方, 5, 0)
ClassRoom(Python機械学習入門, 3, 0)
ClassRoom(機械学習論文輪読­­会(NIPS他) by Team AI 4/10(水), 17, 4)
ClassRoom(音 x 機械学習はどこまで実現できているか？ - 最新理論とユースケース -, 25, 0)
ClassRoom(PyData.Fukui  AI機械学習コラボでコラボ, 4, 4)
ClassRoom(第四陣 信玄パイ「機械学習を業務で利用するのに必要なこと」, 10, 5)
ClassRoom(【無料ワークショップ@大阪】機械学習を成功させるためのデータ前処理と特徴量エンジニアリング（再演）, 66, 11)
ClassRoom(【デモ】iPadで AI:人工知能・ML:機械学習を体験, 20, 7)
ClassRoom([機械学習/DL/TDA]ReNom User Group (RNUG) #11, 80, 49)
ClassRoom(第18回　機械学習超入門, 6, 4)
ClassRoom(第17回　機械学習超入門, 6, 6)
ClassRoom(パターン認識と機械学習(下) by Christopher M. Bishop 輪読会 #11, 12, 8)
ClassRoom(4AI #2 - 機械学習(基礎 & 自然言語処理), 35, 5)


統計も機械学習じゃないか？と思って見てみたが、AI に関連したものは 2 件のみ。  
それを移動すると

1. AI : 52
2. Python : 40
3. デザイン : 19
4. Web : 19
5. 統計 : 11
6. Clowd : 11
7. RPA : 11

In [44]:
for cr in class_rooms:
    if '統計' in cr.title:
        print(cr)

ClassRoom(【Facebookのデータサイエンティスト】が基礎から実務レベルまで教える統計＋R講座, 20, 0)
ClassRoom(『ベイズ統計モデリング―R,JAGS,Stanによるチュートリアル―』読書会 #16, 8, 4)
ClassRoom(【AI】第3回 【統計・情報科学専攻】学生向け ビジネスへのAI適用勉強会 in 名古屋, 20, 0)
ClassRoom(【AI】第1回  【統計・情報科学専攻】学生向け ビジネスへのAI適用勉強会 in 名古屋, 10, 0)
ClassRoom(@大阪淀屋橋開催　第14回 文系のための「統計超入門セミナー」 ～目で見てわかるビジネス統計学, 6, 3)
ClassRoom(【上級者も歓迎】ハンズオンで学ぶ実践的ベイズ統計モデリング【発展編】, 5, 2)
ClassRoom(@大阪淀屋橋開催　第15回 文系のための「統計超入門セミナー」 ～目で見てわかるビジネス統計学, 6, 2)
ClassRoom(@新橋回　第2回 文系のための「統計超入門セミナー」 ～目で見てわかるビジネス統計学～, 6, 3)
ClassRoom(第211回 文系のための「統計超入門セミナー」 ～目で見てわかるビジネス統計学～, 8, 0)
ClassRoom(第212回 文系のための「統計超入門セミナー」 ～目で見てわかるビジネス統計学～, 8, 2)
ClassRoom(第210回 文系のための「統計超入門セミナー」 ～目で見てわかるビジネス統計学～, 8, 1)
ClassRoom(第209回 文系のための「統計超入門セミナー」 ～目で見てわかるビジネス統計学～, 8, 0)
ClassRoom(ゼロからまなぶ統計学(Python編), 10, 0)


### 勉強会のトレンドは…

**機械学習（AI）で決まり** というのは流石に議論の余地はなさそう。  
では隠れた需要はあるのか？

つまり、勉強会自体の開催は少ないが、注目度の高いものはあるのか？

ということで考えてみる。  
これは普通に考えれば、「参加したい人数/募集人数」でスコアリングしてやれば良さそうなのだけど、問題は募集人数不明の奴らである。

これはみてみると結構いる。

単純に、モールとかで開催通知だけする（何人来てもOK）もあれば、単に募集人数書いてなさそうなだけの奴もいる。

In [46]:
for cr in class_rooms:
    if cr.limit == 0:
        print(cr.title)

★第4土曜日・BACHAFIESTA NIGHT
2019年春　M3前日飲み会
2019年春 M3前日練習会
★4/27(土)28(日)29(月祝)東京ベッド『柏工場大開放セール！』
★4/27(土)28(日)29(月祝)フランスベッド・東京工場『ベッド・ソファセール！』
★4/27(土)28(日)29(月祝)カリモク家具鶴見アウトレット『理由(ワケ)ありフェア！』
腹筋さん退職壮行会
★第3土曜日・FIESTAMIGOS NIGHT 
BIT VALLEY -INSIDE- Vol.8
★第2土曜日・ VIENTO SALSA NIGHT
17周年記念イベント　ネットショップカンファレンス2019
ワイワイ麻雀
メーン会場のセルフ誕生日会 〜一斗缶ビリヤニピクニック編〜
★4/13(土)14(日)東京ベッド『柏工場大開放セール！』
OSS活用勉強会 第12回(2019) 勉強会
Arinite | Health and Safety Consultants
アナタのための“トクベツ”なマルシェ！Dear Venus Marche VOL.5 ‐ spring ‐
不祥事対策と「健全に儲け続けるための仕組み」の本質・課題・実効性向上策
監査人のための職業的懐疑心・予兆把握スキルアップ講座
「手づくりマルシェ　ハルフェス2019」
🔸４/７（日）開催！春のそろばんフェスティバル！🔸
★第1土曜日・ NAOBON BACHATA NIGHT
★4/6(土)7(日)フランスベッド・東京工場『ベッド・ソファセール！』
★4/6(土)7(日)カリモク家具鶴見アウトレット『理由(ワケ)ありフェア！』
Agen Poker Online Android
CoderDojo日進 子供向けプログラミング道場　2019/04/27
第9回コーダー道場名護（会場：ヒューマンキャンパス高等学校・名護本校）
第8回コーダー道場名護（会場：名護市マルチメディア館）
BlockChain on the Cloud
第125回オープンソースサロン
Glodiaもくもく会 #9
(2019/4/20PM)第71回オープンCAE勉強会@関西
データサイエンス勉強会 第5回
ALGYAN4周年　IoT祭り2019『IoTとAIとセキュリティ』大講演会＆豪華ノベルティ抽選会！
第16回 #hiro_it（

少し乱暴だけど、平均値でも… *ホワッツ！？*

In [49]:
sum_of_limit = 0
limit_exists = 0

for cr in class_rooms:
    if cr.limit != 0:
        print(f'limit: {cr.limit},  title: {cr.title}')
        limit_exists += 1
        sum_of_limit += cr.limit

avg_limit = sum_of_limit / limit_exists
avg_limit

limit: 5,  title: 4/30 テコンドー練習会 (体験歓迎) ～新しい自分に出会い、自分の可能性を見つけよう～ 【東京・下北沢】
limit: 120,  title: 【4/30パルフェ閉館ラストイベント/全館シェア】寒軍AZITOプレゼンツ！ファイナル祭り！！【ありがとうございました】
limit: 10,  title: 楽園会エンデューロマッドネス体験 in タイ
limit: 10,  title: 2019/04/29 全日本カラオケバトル2020GP 第10回mini予選 兵庫県神戸(カラオケ大会/ボーカルコンテスト)
limit: 10,  title: 2019/04/29 全日本カラオケバトル2020GP 第9回mini予選 大阪府大阪(カラオケ大会/ボーカルコンテスト)
limit: 3,  title: 【初心者限定】Unity１日集中講座〜少人数クラスの１日講習（応用編）
limit: 5,  title: 4/28 テコンドー練習会 (体験歓迎) ～新しい自分に出会い、自分の可能性を見つけよう～ 【東京・下北沢】
limit: 10,  title: 2019/4/28 全日本カラオケバトル2020GP 第8回mini予選 愛知県名古屋(カラオケ大会/ボーカルコンテスト)
limit: 30,  title: ガンオンイベント２０１９ in LFS池袋
limit: 3,  title: 【初心者限定】Unity１日集中講座〜少人数クラスの１日講習（基礎編）
limit: 10,  title: 2019/4/28 全日本カラオケバトル2020GP 第7回mini予選 静岡県浜松(カラオケ大会/ボーカルコンテスト)
limit: 15,  title: 【4/27(土曜日)】第二回English人狼パーティ
limit: 12,  title: 4月27日(土) 第1回 初心者人狼ゲーム会
limit: 20,  title: 練習会 #33
limit: 10,  title: 2019/04/27 全日本カラオケバトル2020GP 第6回mini予選 神奈川県青葉台(カラオケ大会/ボーカルコンテスト)
limit: 3,  title: 女性のフィロソフィー＆心理本の読書会４／２７
limit: 30,  title:

739.3227771010962

どうもエンタメ系のイベントが 10000↑ （最大 55555 人）なんてアホみたいな物を拾ってる。  
流石にフィルタだフィルタ。
2000 人超えたらもう IT 関係無いとみなそう。

In [52]:
sum_of_limit = 0
limit_exists = 0

for cr in class_rooms:
    if cr.limit != 0 and cr.limit < 2000:
        limit_exists += 1
        sum_of_limit += cr.limit

avg_limit = sum_of_limit / limit_exists
avg_limit

32.30929095354523

OK 割と常識的になった。  
では改めて募集人数に対する申し込み人数を計算してみる。

In [56]:
cr_list = []
for cr in class_rooms:
    limit_num = cr.limit if cr.limit > 0 else avg_limit
    cr_list.append([cr.title, float(limit_num), int(cr.joins)])

cr_df = DataFrame(data=cr_list, columns=['title', 'limit', 'joins'])

cr_df['score'] = cr_df['joins'] / cr_df['limit']
cr_df.sort_values(by='score', ascending=False).head(50)

,title,limit,joins,score
359,ALGYAN4周年 IoT祭り2019『IoTとAIとセキュリティ』大講演会＆豪華ノベルティ...,32.309291,261,8.078172
655,現役社員と半日で学ぶ、開発ワークショップ！「JavaScriptでテトリスを作ろう！」の会,4.000000,16,4.000000
554,濱せっく 出張版 #6,45.000000,157,3.488889
730,Vue.jsのサーバーレスアーキテクチャ /メディアサイト制作を進めるために知っておきたいこと,10.000000,30,3.000000
475,LINE Things触ってみようハンズオン,13.000000,32,2.461538
715,いま押さえておくべきJavaScriptの最新機能とは？,50.000000,104,2.080000
861,#エンジニア銭湯,26.000000,53,2.038462
744,Nerima.rb #1,10.000000,20,2.000000
788,SpringBootでマイクロサービスを作ってKubernetesにデプロイしてみる【実践編】,30.000000,58,1.933333
336,CommunitySummitTokyo #3,40.000000,75,1.875000


先頭の奴はなんとも言えないけど、それ以外はわりかし妥当な数字が見えてるように感じる。

* 入門 or ハンズオンが高い人気
* JavaScript 系で新しい路線のものはまだまだ人気（サーバレス/新機能など）。  
  言語としての変化が激しいから、この辺も理解できそう。
* このリスト内では、JavaScript, Python, テスト の関係の単語の出現率がたかそう。  
  Python は脇にに置いといて、JavaScript/テスト 周りは、人気があるのに講座が不足気味と言えそう。

個人的に気になったのが「量子」。  
これ二つだけシレッといるのだけど全体的にはどのくらい出現してるのか

あーうん気のせいだった

In [60]:
cond = cr_df.title.str.contains('量子')
cr_df[cond]

,title,limit,joins,score
272,量子コンピュータハッカソン by Team AI 4/14(日),17.000000,6,0.352941
340,量子コンピューティング勉強会（実践編） #1,12.000000,5,0.416667
387,新しい量子化学―電子構造の理論入門〈上〉輪講その12,32.309291,4,0.123803
650,基礎から応用やトレンドまで学ぶ量子コンピューティング入門セミナー #2,8.000000,4,0.500000
723,量子コンピューターについて語ろうLT大会＆懇親会（2019年4月）,45.000000,34,0.755556
893,量子ゲートで量子アニーリングをプログラミングする,55.000000,66,1.200000
894,量子コンピュータで量子化学を全般的に見返す,55.000000,61,1.109091
